In [9]:
# 출력 피쳐수 계산기

Nin = 255 # 입력 이미지 (255 * 255 * 3 인경우 => 255)
p = 0 # 패딩값 ( 0 , 1 ,)
k = 5 # 필터크기 ( 5 * 5 인경우 => 5)
s = 2 # 스트라이드 값 

Nout = int((Nin + 2*p - k ) / s) + 1

출력필터갯수 = 4
print ( f" Nout = {Nout} * {Nout} * {출력필터갯수}")

 Nout = 126 * 126 * 4


# CNN 

In [8]:
# CNN 

# 라이브러리
from sklearn.model_selection import train_test_split
import numpy as np

import tensorflow as tf
# from tensorflow.keras import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten


# 시드 고정
np.random.seed(42)
tf.random.set_seed(42)

# 데이터 불러오기
from tensorflow.keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# 데이터 전처리 & val 데이터 나누기
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.2)


# 모델 구성 case 01 
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=128,
          validation_data=(X_val, y_val),
          epochs=2)

# 위 모델의 경우 fit 을 하기 전에는 summary()에서 에러가 났음
# 사전 학습 모델을 사용하는 경우(?)에는 안나나?
model.summary() 


# 모델 구성 case 02





# 학습한 모델을 이용하여 평가
model.evaluate(X_test, y_test, verbose=2)

Epoch 1/2


2022-11-26 20:08:14.080875: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-26 20:08:14.511386: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - ETA: 0s - loss: 1.5856 - accuracy: 0.4252

2022-11-26 20:08:25.186041: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 12s 31ms/step - loss: 1.5856 - accuracy: 0.4252 - val_loss: 1.3421 - val_accuracy: 0.5300
Epoch 2/2
313/313 [==============================] - 9s 28ms/step - loss: 1.1964 - accuracy: 0.5755 - val_loss: 1.0989 - val_accuracy: 0.6135
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 conv2d_13 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 8, 8, 64)         0         
 2D)                                       

# Transformer 

In [3]:
# Transformer ( VGG16 )
from sklearn.model_selection import train_test_split
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D


# 시드 고정
np.random.seed(42)
tf.random.set_seed(42)


# 데이터 로딩
from tensorflow.keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# 전처리
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

# val data 나누기
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.2)


## 사전 학습 모델 로딩
from tensorflow.keras.applications.vgg16 import VGG16
pretrained_model = VGG16(weights='imagenet', include_top=False)

# 모델 구성
model = Sequential()
model.add(pretrained_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(128,activation='relu'))
model.add(Dense(10,activation='softmax'))

model.summary()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=128,
          validation_data=(X_val, y_val),
          epochs=10)


# 학습한 모델을 이용하여 평가
model.evaluate(X_test, y_test, verbose=2)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 global_average_pooling2d_1   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 128)               65664     
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 14,781,642
Trainable params: 14,781,642
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


2022-11-29 07:46:30.892409: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-29 07:46:31.456169: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - ETA: 0s - loss: 1.9556 - accuracy: 0.2228

2022-11-29 07:47:54.103837: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 89s 270ms/step - loss: 1.9556 - accuracy: 0.2228 - val_loss: 1.6967 - val_accuracy: 0.3338
Epoch 2/10
313/313 [==============================] - 78s 248ms/step - loss: 1.4453 - accuracy: 0.4412 - val_loss: 1.2798 - val_accuracy: 0.5213
Epoch 3/10
198/313 [=================>............] - ETA: 25s - loss: 1.1327 - accuracy: 0.5827